<div style="text-align:center;"><img src="http://www.mf-data-science.fr/images/projects/intro.jpg" style='width:100%; margin-left: auto; margin-right: auto; display: block;' /></div>

# <span style="color: #641E16">Contexte</span>
Nous allons ici développer un algorithme de Machine Learning destiné à assigner automatiquement plusieurs tags pertinents à une question posée sur le célébre site Stack overflow.     
Ce programme s'adresse principalement aux nouveaux utilisateurs, afin de leur suggérer quelques tags relatifs à la question qu'ils souhaitent poser.

### Les données sources
Les données ont été cleanées dans le Notebook Kaggle [Stackoverflow questions - data cleaning](https://www.kaggle.com/michaelfumery/stackoverflow-questions-data-cleaning). Dans ce nettoyage ont par exemple été appliquées les techniques de stop words, suppression de la ponctuation et des liens, tokenisation, lemmatisation ...

### Objectif de ce Notebook
Dans ce Notebook, nous allons traiter la partie **modélisation des données textuelles avec des modèles supervisés et non supervisés**.     

Tous les Notebooks du projet seront **versionnés dans Kaggle mais également dans un repo GitHub** disponible à l'adresse https://github.com/MikaData57/Analyses-donnees-textuelles-Stackoverflow

# <span style="color:#641E16">Sommaire</span>
1. [Preprocessing : Bag of Words / Tf-Idf](#section_1)
2. [Modèles non supervisés](#section_2)     
    2.1. [Modèle LDA](#section_2_1)     
    2.2. [Modèle NMF](#section_2_2)     

In [ ]:
# Install package for PEP8 verification
!pip install pycodestyle
!pip install --index-url https://test.pypi.org/simple/ nbpep8

In [ ]:
# Import Python libraries
import os
import warnings
import time
import numpy as np
import pandas as pd
from ast import literal_eval
import re
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.model_selection import GridSearchCV
import gensim
from scipy.sparse import hstack
import pyLDAvis
import pyLDAvis.sklearn
import pyLDAvis.gensim_models as gensimvis

# Library for PEP8 standard
from nbpep8.nbpep8 import pep8

In [ ]:
warnings.filterwarnings('ignore', category=DeprecationWarning)
plt.style.use('seaborn-whitegrid')
sns.set_style("whitegrid")

In [ ]:
# Define path to data
path = '../input/stackoverflow-questions-filtered-2011-2021/'
data = pd.read_csv(path+"StackOverflow_questions_2009_2020_cleaned.csv",
                   sep=";", index_col=0,
                   converters={"Title": literal_eval,
                               "Body": literal_eval,
                               "Tags": literal_eval})
data.head(3)

In [ ]:
data.shape

Nous allons également créer une variable `Full_doc` qui accueillera le document complet de chaque item (Title et Body) :

In [ ]:
data["Full_doc"] = data["Title"] + data["Body"]
data["Full_doc"].head(3)

# <span style="color:#641E16" id="section_1">Preprocessing : Bag of Words / Tf-Idf</span>

Pour alimenter les modèles de machine learning, nous avons besoin de traiter des données numériques. Le modèle **Bag of Words** apprend un vocabulaire à partir de tous les documents, puis modélise chaque document en comptant le nombre de fois où chaque mot apparaît, convertissant donc les données textuelles en données numériques.

Nos données ayant déjà été cleanées et tokenisées dans le Notebook [stackoverflow-questions-data-cleaning](https://www.kaggle.com/michaelfumery/stackoverflow-questions-data-cleaning), nous allons initialiser l'algorithme du `CountVectorizer` sur les variables `Title` et `Body` *(X1 et X2)* sans preprocessing. Enfin, nous allons utiliser le module `TfidfVectorizer` de la librairie Scikit-Learn pour combiner le `CountVectorizer` et `TfidfTransformer`. Cela aura pour effet de pondérer la fréquence d'apparition des par un indicateur de similarité *(si ce mot est commun ou rare dans tous les documents)*. Dans cette partie, nous allons **éliminer les mots qui apparaissent dans plus de 60% des documents** (`max_df = 0.6`).

la métrique tf-idf ***(Term-Frequency - Inverse Document Frequency)*** utilise comme indicateur de similarité l'inverse document frequency qui est l'inverse de la proportion de document qui contient le terme, à l'échelle logarithmique.

Pour préparer nos targets *(pour les modèles supervisés)*, nous allons utiliser `MultiLabelBinarizer` de Scikit-Learn puisque nos `Tags` sont multiples.

In [ ]:
# Initialize the "CountVectorizer" object for Title, Body and Full_doc
title_vectorizer = TfidfVectorizer(analyzer="word",
                                   max_df=.6,
                                   min_df=0.005,
                                   tokenizer=None,
                                   preprocessor=' '.join,
                                   stop_words=None,
                                   lowercase=False)

body_vectorizer = TfidfVectorizer(analyzer="word",
                                  max_df=.6,
                                  min_df=0.005,
                                  tokenizer=None,
                                  preprocessor=' '.join,
                                  stop_words=None,
                                  lowercase=False)

full_vectorizer = TfidfVectorizer(analyzer="word",
                                  max_df=.6,
                                  min_df=0.005,
                                  tokenizer=None,
                                  preprocessor=' '.join,
                                  stop_words=None,
                                  lowercase=False)

Xt_tfidf = title_vectorizer.fit_transform(data["Title"])
Xb_tfidf = body_vectorizer.fit_transform(data["Body"])
X_tfidf = full_vectorizer.fit_transform(data["Full_doc"])

print("Shape of X for Title: {}".format(Xt_tfidf.shape))
print("Shape of X for Body: {}".format(Xb_tfidf.shape))
print("Shape of X for Full_doc: {}".format(X_tfidf.shape))

# Multilabel binarizer for targets
multilabel_binarizer = MultiLabelBinarizer() 
y = multilabel_binarizer.fit_transform(data["Tags"])
print("Shape of y: {}".format(y.shape))

Nous avons à présent une matrice de 30846 lignes et 4306 colonnes qui sera interprétable par nos modèles de Machine Learning. Comme les matrices sont relativment importantes, nous allons **vérifier le nombre de cellules qui ne sont pas à 0** :

In [ ]:
title_dense = Xt_tfidf.todense()
body_dense = Xb_tfidf.todense()
full_dense = X_tfidf.todense()
print("Title sparsicity: {:.3f} %"\
      .format(((title_dense > 0).sum()/title_dense.size)*100))
print("Body sparsicity: {:.3f} %"\
      .format(((body_dense > 0).sum()/body_dense.size)*100))
print("Full_doc sparsicity: {:.3f} %"\
      .format(((full_dense > 0).sum()/full_dense.size)*100))

Nous constatons que cette mesure est meilleure pour la varaible englobant Title et Body *(Full_doc)*.

# <span style="color:#641E16" id="section_2">Modèles non supervisés</span>

## <span id="section_2_1">Modèle LDA</span>
LDA, ou **Latent Derelicht Analysis** est un modèle probabiliste qui, pour obtenir des affectations de cluster, utilise deux valeurs de probabilité : $P(word | topics)$ et $P(topics | documents)$. Ces valeurs sont calculées sur la base d'une attribution aléatoire initiale, puis le calcul est répété pour chaque mot dans chaque document, pour décider de leur attribution de sujet. Dans cette méthode itérative, ces probabilités sont calculées plusieurs fois, jusqu'à la convergence de l'algorithme.

Nous allons entrainer 3 modèles séparement pour le `Title`, le `Body` et `Full_doc` :

In [ ]:
print("-"*50)
print("Start LDA fitting on Title ...")
print("-" * 50)
start_time = time.time()
# Initializing the LDA on Title
title_lda = LatentDirichletAllocation(n_components=20,
                                      random_state=8,
                                      n_jobs=-1)

# Fit LDA on Title
title_lda.fit(Xt_tfidf)

exec_time = time.time() - start_time
print("End of training :")
print("Execution time : {:.2f}s".format(exec_time))
print("-" * 50)

### <span style="color:#641E16;">Visualisation des résultats de LDA Sklearn sur Title</span>

In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(title_lda, 
                         Xt_tfidf, 
                         title_vectorizer)

In [ ]:
print("-"*50)
print("Start LDA fitting on Body ...")
print("-" * 50)
start_time = time.time()
# Initializing the LDA
body_lda = LatentDirichletAllocation(n_components=20,
                                     random_state=8,
                                     n_jobs=-1)

# Fit LDA on Body
body_lda.fit(Xb_tfidf)

exec_time = time.time() - start_time
print("End of training :")
print("Execution time : {:.2f}s".format(exec_time))
print("-" * 50)

### <span style="color:#641E16;">Visualisation des résultats de LDA Sklearn sur Body</span>

In [ ]:
pyLDAvis.sklearn.prepare(body_lda, 
                         Xb_tfidf, 
                         body_vectorizer)

In [ ]:
print("-"*50)
print("Start LDA fitting on Full_doc ...")
print("-" * 50)
start_time = time.time()
# Initializing the LDA
full_lda = LatentDirichletAllocation(n_components=20,
                                     random_state=8,
                                     n_jobs=-1)

# Fit LDA on Full_doc
full_lda.fit(X_tfidf)

exec_time = time.time() - start_time
print("End of training :")
print("Execution time : {:.2f}s".format(exec_time))
print("-" * 50)

### <span style="color:#641E16;">Visualisation des résultats de LDA Sklearn sur Full_doc</span>

In [ ]:
pyLDAvis.sklearn.prepare(full_lda, 
                         X_tfidf, 
                         full_vectorizer)

Que ce soit avec la variable `Title` ou avec la variable `Body`, il semble très **difficile de "nommer" les topics créés car les mots qui les composent sont très variés et sans réél fil conducteur**. LDA sur le document complet `Full_doc`, englobant les tokens de Title et Body semble donner des résultats plus cohérents.

Cependant, dans ces algoritmes LDA, nous avons fixé arbitrairement le paramètre `n_components` qui représente le nombre de topics à créer. Afin de sélectionner le meilleur nombre de topics à créer, nous allons utiliser une `GridSearch` avec Cross-validation et testerons 2 métriques :
- Log likelihood : Densité de vraisemblance 
- Perplexity : $(\exp(-1 \times \text{log-likelihood})$

Regardons déjà ces métriques obtenues sur nos premiers modèles :

In [ ]:
# Log Likelihood
print("-"*50)
print("Log Likelihood")
print("-" * 50)
print("Log Likelihood on Title Model : {}".format(title_lda.score(Xt_tfidf)))
print("Log Likelihood on Body Model : {}".format(body_lda.score(Xb_tfidf)))
print("Log Likelihood on Full_doc Model : {}".format(full_lda.score(X_tfidf)))

# Perplexity
print("-"*50)
print("Perplexity")
print("-" * 50)
print("Perplexity on Title Model : {}".format(title_lda.perplexity(Xt_tfidf)))
print("Perplexity on Body Model : {}".format(body_lda.perplexity(Xb_tfidf)))
print("Perplexity on Full_doc Model : {}".format(full_lda.perplexity(X_tfidf)))

### Amélioration des paramètres :
Un modèle avec une log-likelihood plus élevée et une perplexité plus faible est considéré comme bon. Ici, on remarque que **notre modèle ne semble pas très performant**.
Appliquons maintenant une itération sur le nombre de topics de notre LDA avec la variable `Full_doc` :

In [ ]:
kmin, kmax = 1, 100
topic_models = []

for k in np.arange(kmin, kmax, 10):
    print("-"*50)
    print("Applying LDA for k = {}".format(k))
    print("-"*50)
    # Run LDA
    model = LatentDirichletAllocation(n_components=k,
                                      random_state=8,
                                      n_jobs=-1)
    W = model.fit_transform(X_tfidf)
    H = model.components_
    loglikelihood = model.score(X_tfidf)
    perplexity = model.perplexity(X_tfidf)
    topic_models.append((k, W, H, loglikelihood, perplexity))
    print("Indicative Loglikelihood = {:.2f}".format(loglikelihood))

In [ ]:
topic_models = pd.DataFrame(topic_models,
                            columns=["N_components", "X_Transform",
                                     "Model_component",
                                     "Loglikelihood", "Perplexity"])

Affichons les scores des divers modèle :

In [ ]:
# Show graph
fig, ax1 = plt.subplots(figsize=(12, 8))
x = topic_models['N_components']
y1 = topic_models['Loglikelihood']
ax1.plot(x, y1, label='Loglikelihood score')
ax1.axvline(x=1, color='r', alpha=.7,
            linestyle='dashdot', label='Best param')
ax1.set_xlabel("Number of topics")
ax1.set_ylabel("Loglikelihood Score")

ax2 = ax1.twinx()
y2 = topic_models['Perplexity']
ax2.plot(x, y2, label='Perplexity score',
         color='g', alpha=.5,
         linestyle='--',)
ax2.set_ylabel("Perplexity score")

plt.title("Choosing Optimal LDA Model\n",
          color="#641E16", fontsize=18)
legend = fig.legend(loc=1, bbox_to_anchor=(.92, .9))

fig.tight_layout()
plt.show()

On remarque ici que la modélisation non supervisée avec LDA n'est pas adaptée. En effet, le meilleur nombre de topics se situerait à 1, ce qui signifie que **l'algorithme ne parvient pas a établir de groupes bien distincts**.

Nous allons donc tester une seconde modélisation non supervisée.

## <span id="section_2_2">Modèle NMF</span>
<details>
  <summary style="color:blue;">Explication du modèle</summary>
  
  ## NMF
  La factorisation matricielle non négative *(**N**on-negative **M**atrix **F**actorization)* est un modèle linéaire-algéabrique, qui factorise des vecteurs de grande dimension dans une représentation de faible dimension. Similaire à l'analyse en composantes principales *(PCA)*, NMF profite du fait que **les vecteurs sont non négatifs**. En les factorisant dans la forme de dimension inférieure, NMF force les coefficients à être également non négatifs.<br/><br/>
    Prenons une matrice d'origine $A$, nous pouvons obtenir deux matrices $W$ et $H$, telles que $A = WH$. NMF a une propriété de clustering, telle que $W$ et $H$ représentent les informations suivantes sur $A$ :
    <ul><li>$A$ (Matrice Document-word) : Matrice qui contient "quels mots apparaissent dans quels documents".</li>
    <li>$W$ (Vecteurs de base) : Topics découverts à partir des documents.</li>
    <li>$H$ (Matrice de coefficients) : les poids pour les topics dans chaque document.</li></ul><br/>
    Nous calculons $W$ et $H$ en optimisant sur une **fonction objectif**, en mettant à jour à la fois $W$ et $H$ de manière itérative jusqu'à convergence.</br></br>
    $$\large \frac{1}{2} ||A - WH||^2_F = \sum_{i=1}^{n} \sum_{j=1}^{m} (A_{ij} - (WH)_{ij})^2$$</br>
    Dans cette fonction objectif, nous mesurons l'erreur de reconstruction entre A et le produit de ses facteurs W et H, en fonction de la distance euclidienne. Les valeurs mises à jour sont calculées dans des opérations parallèles, et en utilisant les nouveaux W et H, nous recalculons l'erreur de reconstruction, en répétant ce processus jusqu'à la convergence.
</details>

Le modèle **NMF ne peut malheureusement pas être scoré** dans une GridSearch. Nous allons donc nous baser sur les résultats de la GridSearch LDA pour déterminer un nombre correct de composants. Ici, nous prendrons **10 topics** pour avoir un bon compromis "temps d'entrainement" / précision.

In [ ]:
print("-"*50)
print("Start NMF fitting on Title ...")
print("-" * 50)
start_time = time.time()
# Initializing the NMF
title_nmf = NMF(n_components=10,
                init='nndsvd',
                random_state=8)

# Fit NMF on Title vectorized
title_nmf.fit(Xt_tfidf)

exec_time = time.time() - start_time
print("End of training :")
print("Execution time : {:.2f}s".format(exec_time))
print("-" * 50)

L'entrainement est plus rapide que pour LDA. Visualisons à présent les résulats en plotant **les 20 meilleurs mots de chaque topic**.

In [ ]:
def plot_top_words(model, feature_names, n_top_words, title):
    """Function for displaying the plots of the 
    best x words representative of the categories of NMF.

    Parameters
    ----------------------------------------
    model : NMF model
        Fitted model of NMF to plot
    feature_names : array
        Categories result of the vectorizer (TFIDF ...)
    n_top_words : int
        Number of words for each topic.
    title : string
        Title of the plot.
    ----------------------------------------
    """
    fig, axes = plt.subplots(2, 5, figsize=(30, 20), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
        top_features = [feature_names[i] for i in top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        bartopic = ax.barh(top_features, weights, height=0.7)
        bartopic[0].set_color('#f48023')
        ax.set_title(f'Topic {topic_idx +1}',
                     fontdict={'fontsize': 30})
        ax.invert_yaxis()
        ax.tick_params(axis='both', which='major', labelsize=20)
        for i in 'top right left'.split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=36, color="#641E16")

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.show()

In [ ]:
tf_feature_names = title_vectorizer.get_feature_names()
plot_top_words(title_nmf, tf_feature_names, 20, 'Topics in NMF model for Title feature')

**La modélisation avec NMF nous apporte des catégories plus lisibles que celles de l'algorithme LDA**. 1 mot est toujours plus représentaif de cette catégorie mais les regroupements sont globalement cohérents. Le topic 4 par exemple illustre bien les sujets liés à Python *(on y retrouve des mots comme Numpy, package par exemple)*. Le topic 6 traite les sujets liés à Android et le topic 1 semble traiter des sujets web.

Nous allons **tester cette modélisation sur la variable Body** cette fois :

In [ ]:
print("-"*50)
print("Start NMF fitting on Body ...")
print("-" * 50)
start_time = time.time()
# Initializing the NMF
body_nmf = NMF(n_components=10,
               init='nndsvd',
               random_state=8)

# Fit NMF on Body vectorized
body_nmf.fit(Xb_tfidf)

exec_time = time.time() - start_time
print("End of training :")
print("Execution time : {:.2f}s".format(exec_time))
print("-" * 50)

bf_feature_names = body_vectorizer.get_feature_names()
plot_top_words(body_nmf, bf_feature_names, 30, 'Topics in NMF model for Body feature')

Basé sur le corpus Body, le modèle fait sortir des thèmes différents, plus génériques. Le topic 2 par exemple traite des sujets liés à l'audio; le topic 3 des sujets liés à l'image et le topic 9 des sujets plus hardware ...
**NMF est également beaucoup plus rapide que LDA**, ce qui est un atout pour en terme de puissance de calcul nécessaire.

In [ ]:
print("-"*50)
print("Start NMF fitting on Full_doc ...")
print("-" * 50)
start_time = time.time()
# Initializing the NMF
full_nmf = NMF(n_components=10,
               init='nndsvd',
               random_state=8)

# Fit NMF on Body vectorized
full_nmf.fit(X_tfidf)

exec_time = time.time() - start_time
print("End of training :")
print("Execution time : {:.2f}s".format(exec_time))
print("-" * 50)

ff_feature_names = full_vectorizer.get_feature_names()
plot_top_words(full_nmf, ff_feature_names, 30, 'Topics in NMF model for Full_doc')